In [1]:
### Config
### Import dependencies
# Dependencies for data manipulation
import pandas as pd
import numpy as np
import os
from datetime import datetime, date
from dateutil.relativedelta import *

# Dependencies for Databases
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
from sqlalchemy import create_engine

# Find environment variables
DATABASE_URL = os.environ.get("DATABASE_URL", None)
# sqlalchemy deprecated urls which begin with "postgres://"; now it needs to start with "postgresql://"
if DATABASE_URL.startswith("postgres://"):
    DATABASE_URL = DATABASE_URL.replace("postgres://", "postgresql://", 1)

In [79]:
### Build Postgresql table for historical player stats
# Get historical data
histdf = pd.read_csv('onetimeData/modelSource.csv', index_col=0)

# Select only previous two years
histdf = histdf.loc[histdf['season']>=2020]

# Clean defense names in historical data
histdf['player'] = histdf['player'].str.upper()
# Drop punctuation
histdf['player'] = histdf['player'].str.replace(".", "")
histdf['player'] = histdf['player'].str.replace(",", "")
histdf['player'] = histdf['player'].str.replace("'", "")

histdf = histdf[[
    'season', 'week', 
    'team', 'opponent',
    'player', 'pos', 'posRank', 
    'passA', 'passC', 'passY', 'passT', 'passI', 'pass2', 
    'rushA', 'rushY', 'rushT', 'rush2', 
    'recC', 'recY', 'recT', 'rec2', 'fum', 
    'XPA', 'XPM', 'FGA', 'FGM', 'FG50', 
    'defSack', 'defI', 'defSaf', 'defFum', 'defBlk', 'defT', 'defPtsAgainst', 'defPassYAgainst', 'defRushYAgainst', 'defYdsAgainst']]


/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # This is added back by InteractiveShellApp.init_path()


In [80]:

# Save space by creating a yearly summary for historical data
## histdf = histdf.loc[histdf.season>=2020]
# histdf_avg = histdf.groupby(['season', 'player']).mean()
# histdf_avg['gamesPlayed'] = histdf.groupby(['season', 'player'])['week'].count()
# histdf_avg.drop(columns='week', inplace=True)
# gamesPlayedCol = histdf_avg.pop('gamesPlayed')
# histdf_avg.insert(0, 'gamesPlayed', gamesPlayedCol)
# histdf = histdf_avg
# histdf.reset_index(inplace=True)


In [81]:
# Find average stats for 1 season prior
prior1 = histdf.loc[histdf.season==2021]
prior1_avg = prior1.groupby(['player']).mean()
prior1_avg['gamesPlayed'] = prior1.groupby(['player'])['week'].count()
prior1_avg.drop(columns=['week', 'season'], inplace=True)
gamesPlayedCol = prior1_avg.pop('gamesPlayed')
prior1_avg.insert(0, 'gamesPlayed', gamesPlayedCol)
prior1 = prior1_avg
prior1.reset_index(inplace=True)
prior1


,player,gamesPlayed,passA,passC,passY,passT,passI,pass2,rushA,rushY,...,defSack,defI,defSaf,defFum,defBlk,defT,defPtsAgainst,defPassYAgainst,defRushYAgainst,defYdsAgainst
0,AARON JONES,15,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,11.400000,53.266667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AARON RODGERS,15,34.20,23.466667,265.133333,2.333333,0.266667,0.000000,2.133333,6.266667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ADAM HUMPHRIES,16,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ADAM PRENTICE,3,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,1.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ADAM SHAHEEN,6,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,ZACH PASCAL,14,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,1.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
661,ZACH WILSON,12,30.25,17.166667,187.250000,0.666667,0.916667,0.083333,2.250000,13.416667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
662,ZACK MOSS,12,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,7.583333,28.083333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
663,ZANE GONZALEZ,12,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
# Find average stats for 2 seasons prior
prior2 = histdf.loc[histdf.season>=2020]
prior2_avg = prior2.groupby(['player']).mean()
prior2_avg['gamesPlayed'] = prior2.groupby(['player'])['week'].count()
prior2_avg.drop(columns=['week', 'season'], inplace=True)
gamesPlayedCol = prior2_avg.pop('gamesPlayed')
prior2_avg.insert(0, 'gamesPlayed', gamesPlayedCol)
prior2 = prior2_avg
prior2.reset_index(inplace=True)
prior2

,player,gamesPlayed,passA,passC,passY,passT,passI,pass2,rushA,rushY,...,defSack,defI,defSaf,defFum,defBlk,defT,defPtsAgainst,defPassYAgainst,defRushYAgainst,defYdsAgainst
0,AARON JONES,29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.827586,65.620690,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AARON RODGERS,31,33.516129,23.354839,266.967742,2.677419,0.290323,0.000000,2.258065,7.838710,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ADAM HUMPHRIES,22,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ADAM PRENTICE,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,1.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ADAM SHAHEEN,15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817,ZACH PASCAL,30,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.700000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
818,ZACH WILSON,12,30.250000,17.166667,187.250000,0.666667,0.916667,0.083333,2.250000,13.416667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
819,ZACK MOSS,25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.120000,32.720000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
820,ZANE GONZALEZ,24,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
# Send dfs to database
# Prepare prior1 df
# Build the SQL query that will list columns and datatypes
string1 = [x + " VARCHAR(32)" for x in prior1.columns[:1]] + [
    x + " SMALLINT" for x in prior1.columns[1:2]] + [
    x + " FLOAT(8)" for x in prior1.columns[2:]] 
string1 = str(string1)
string1 = string1.replace("'", "")
string1 = string1.replace("[", "")
string1 = string1.replace("]", "")
#print(f'CREATE TABLE IF NOT EXISTS prior1({string1})')

# Write the df to the Postgresql database
try:
    # connect to database
    conn = psycopg2.connect(DATABASE_URL, sslmode='require')
    engine = create_engine(DATABASE_URL)
    cursor = conn.cursor()
    # Create table for prior1
    cursor.execute(f'CREATE TABLE IF NOT EXISTS prior1({string1})')
    conn.commit()
    # Populate table with data
    prior1.to_sql('prior1', engine, if_exists='replace', index = False)
except Exception as error:
    print(error)
finally:
    if conn:
        cursor.close()
        conn.close()

In [85]:
# Send dfs to database
# Prepare prior2 df
# Build the SQL query that will list columns and datatypes
string1 = [x + " VARCHAR(32)" for x in prior2.columns[:1]] + [
    x + " SMALLINT" for x in prior2.columns[1:2]] + [
    x + " FLOAT(8)" for x in prior2.columns[2:]] 
string1 = str(string1)
string1 = string1.replace("'", "")
string1 = string1.replace("[", "")
string1 = string1.replace("]", "")
#print(f'CREATE TABLE IF NOT EXISTS prior2({string1})')

# Write the df to the Postgresql database
try:
    # connect to database
    conn = psycopg2.connect(DATABASE_URL, sslmode='require')
    engine = create_engine(DATABASE_URL)
    cursor = conn.cursor()
    # Create table for prior2
    cursor.execute(f'CREATE TABLE IF NOT EXISTS prior2({string1})')
    conn.commit()
    # Populate table with data
    prior2.to_sql('prior2', engine, if_exists='replace', index = False)
except Exception as error:
    print(error)
finally:
    if conn:
        cursor.close()
        conn.close()